In [ ]:
import pygame 
# For Windows: pip install -U pygame --user
# https://www.pygame.org/wiki/about

In [ ]:
# Define some colors
BLACK    = (   0,   0,   0)
WHITE    = ( 255, 255, 255)

# This is a simple class that will help us print to the screen
# It has nothing to do with the joysticks, just outputting the
# information.
class TextPrint:
    def __init__(self):
        self.reset()
        self.font = pygame.font.Font(None, 20)

    def print(self, screen, textString):
        textBitmap = self.font.render(textString, True, BLACK)
        screen.blit(textBitmap, [self.x, self.y])
        self.y += self.line_height
        
    def reset(self):
        self.x = 10
        self.y = 10
        self.line_height = 15
        
    def indent(self):
        self.x += 10
        
    def unindent(self):
        self.x -= 10
    

pygame.init()
 
# Set the width and height of the screen [width,height]
size = [500, 700]
screen = pygame.display.set_mode(size)

pygame.display.set_caption("My Game")

#Loop until the user clicks the close button.
done = False

# Used to manage how fast the screen updates
clock = pygame.time.Clock()

# Initialize the joysticks
pygame.joystick.init()
    
# Get ready to print
textPrint = TextPrint()

# -------- Main Program Loop -----------
while done==False:
    # EVENT PROCESSING STEP
    for event in pygame.event.get(): # User did something
        if event.type == pygame.QUIT: # If user clicked close
            done=True # Flag that we are done so we exit this loop
        
        # Possible joystick actions: JOYAXISMOTION JOYBALLMOTION JOYBUTTONDOWN JOYBUTTONUP JOYHATMOTION
        if event.type == pygame.JOYBUTTONDOWN:
            print("Joystick button pressed.")
        if event.type == pygame.JOYBUTTONUP:
            print("Joystick button released.")
            
 
    # DRAWING STEP
    # First, clear the screen to white. Don't put other drawing commands
    # above this, or they will be erased with this command.
    screen.fill(WHITE)
    textPrint.reset()

    # Get count of joysticks
    joystick_count = pygame.joystick.get_count()

    textPrint.print(screen, "Number of joysticks: {}".format(joystick_count) )
    textPrint.indent()
    
    # For each joystick:
    for i in range(joystick_count):
        joystick = pygame.joystick.Joystick(i)
        joystick.init()
    
        textPrint.print(screen, "Joystick {}".format(i) )
        textPrint.indent()
    
        # Get the name from the OS for the controller/joystick
        #name = joystick.get_name()
        #textPrint.print(screen, "Joystick name: {}".format(name) )
        
        # Usually axis run in pairs, up/down for one, and left/right for
        # the other.
        axes = joystick.get_numaxes()
        textPrint.print(screen, "Number of axes: {}".format(axes) )
        textPrint.indent()
        
        for i in range( axes ):
            axis = joystick.get_axis( i )
            textPrint.print(screen, "Axis {} value: {:>6.3f}".format(i, axis) )
        textPrint.unindent()
            
        buttons = joystick.get_numbuttons()
        textPrint.print(screen, "Number of buttons: {}".format(buttons) )
        textPrint.indent()

        for i in range( buttons ):
            button = joystick.get_button( i )
            textPrint.print(screen, "Button {:>2} value: {}".format(i,button) )
        textPrint.unindent()
        
        # Hat switch. All or nothing for direction, not like joysticks.
        # Value comes back in an array.
        hats = joystick.get_numhats()
        textPrint.print(screen, "Number of hats: {}".format(hats) )
        textPrint.indent()

        for i in range( hats ):
            hat = joystick.get_hat( i )
            textPrint.print(screen, "Hat {} value: {}".format(i, str(hat)) )
        textPrint.unindent()
        
        textPrint.unindent()

    
    # ALL CODE TO DRAW SHOULD GO ABOVE THIS COMMENT
    
    # Go ahead and update the screen with what we've drawn.
    pygame.display.flip()

    # Limit to 20 frames per second
    clock.tick(20)
    
# Close the window and quit.
# If you forget this line, the program will 'hang'
# on exit if running from IDLE.
pygame.quit ()

In [ ]:
import sys
import ctypes
from sdl2 import *

def main():
    SDL_Init(SDL_INIT_VIDEO)
    window = SDL_CreateWindow(b"Hello World",
                              SDL_WINDOWPOS_CENTERED, SDL_WINDOWPOS_CENTERED,
                              592, 460, SDL_WINDOW_SHOWN)
    windowsurface = SDL_GetWindowSurface(window)

    image = SDL_LoadBMP(b"exampleimage.bmp")
    SDL_BlitSurface(image, None, windowsurface, None)

    SDL_UpdateWindowSurface(window)
    SDL_FreeSurface(image)

    running = True
    event = SDL_Event()
    while running:
        while SDL_PollEvent(ctypes.byref(event)) != 0:
            if event.type == SDL_QUIT:
                running = False
                break

    SDL_DestroyWindow(window)
    SDL_Quit()
    return 0

if __name__ == "__main__":
    sys.exit(main())

In [1]:
#Feedback/Vibration/Haptic effects
import sdl2 #Local sdl2 dir which is a wrapper to SDL2.dll
class Haptic:

    def __init__(self):
        sdl2.SDL_Init(sdl2.SDL_INIT_TIMER | sdl2.SDL_INIT_JOYSTICK | sdl2.SDL_INIT_HAPTIC)

        print("Trying to find haptics")
        if (sdl2.SDL_NumHaptics() == 0):
            print("No haptic devices found")
            sdl2.SDL_Quit()
            exit(0)

        for index in range(0, sdl2.SDL_NumHaptics()):
	        print("Found", index, ":", sdl2.SDL_HapticName(index))
    
        index = 0

        self.haptic = sdl2.SDL_HapticOpen(index)
        if self.haptic == None:
            print("Unable to open device")
            sdl2.SDL_Quit()
            exit(0)
        else:
            print("Using device", index)

        self.nefx = 0
        self.efx = [0] * 12
        self.id = [0] * 12
        supported = sdl2.SDL_HapticQuery(self.haptic)
    
        if (supported & sdl2.SDL_HAPTIC_SINE):
            print("   effect", self.nefx, "Sine Wave")
            self.efx[self.nefx] = sdl2.SDL_HapticEffect(type=sdl2.SDL_HAPTIC_SINE, periodic= \
                sdl2.SDL_HapticPeriodic(type=sdl2.SDL_HAPTIC_SINE, direction=sdl2.SDL_HapticDirection(type=sdl2.SDL_HAPTIC_POLAR, dir=(9000,0,0)), \
                period=1000, magnitude=0x4000, length=5000, attack_length=1000, fade_length=1000))
            self.id[self.nefx] = sdl2.SDL_HapticNewEffect(self.haptic, self.efx[self.nefx])
            self.nefx += 1

        if (supported & sdl2.SDL_HAPTIC_TRIANGLE):
            print("   effect", self.nefx, "Triangle")
            self.efx[self.nefx] = sdl2.SDL_HapticEffect(type=sdl2.SDL_HAPTIC_TRIANGLE, periodic= \
                sdl2.SDL_HapticPeriodic(type=sdl2.SDL_HAPTIC_SINE, direction=sdl2.SDL_HapticDirection(type=sdl2.SDL_HAPTIC_CARTESIAN, dir=(1,0,0)), \
                period=1000, magnitude=0x4000, length=5000, attack_length=1000, fade_length=1000))
            self.id[self.nefx] = sdl2.SDL_HapticNewEffect(self.haptic, self.efx[self.nefx])
            self.nefx += 1

        if (supported & sdl2.SDL_HAPTIC_SAWTOOTHUP):
            print("   effect", self.nefx, "Sawtooth Up")
            self.efx[self.nefx] = sdl2.SDL_HapticEffect(type=sdl2.SDL_HAPTIC_SAWTOOTHUP, periodic= \
                sdl2.SDL_HapticPeriodic(type=sdl2.SDL_HAPTIC_SAWTOOTHUP, direction=sdl2.SDL_HapticDirection(type=sdl2.SDL_HAPTIC_POLAR, dir=(9000,0,0)), \
                period=500, magnitude=0x5000, length=5000, attack_length=1000, fade_length=1000))
            self.id[self.nefx] = sdl2.SDL_HapticNewEffect(self.haptic, self.efx[self.nefx])
            self.nefx += 1

        if (supported & sdl2.SDL_HAPTIC_SAWTOOTHDOWN):
            print("   effect", self.nefx, "Sawtooth Down")
            self.efx[self.nefx] = sdl2.SDL_HapticEffect(type=sdl2.SDL_HAPTIC_SAWTOOTHDOWN, periodic= \
                sdl2.SDL_HapticPeriodic(type=sdl2.SDL_HAPTIC_SAWTOOTHDOWN, direction=sdl2.SDL_HapticDirection(type=sdl2.SDL_HAPTIC_CARTESIAN, dir=(1,0,0)), \
                period=500, magnitude=0x5000, length=5000, attack_length=1000, fade_length=1000))
            self.id[self.nefx] = sdl2.SDL_HapticNewEffect(self.haptic, self.efx[self.nefx])
            self.nefx += 1

        if (supported & sdl2.SDL_HAPTIC_RAMP):
            print("   effect", self.nefx, "Ramp")
            self.efx[self.nefx] = sdl2.SDL_HapticEffect(type=sdl2.SDL_HAPTIC_RAMP, ramp= \
                sdl2.SDL_HapticRamp(type=sdl2.SDL_HAPTIC_RAMP, direction=sdl2.SDL_HapticDirection(type=sdl2.SDL_HAPTIC_POLAR, dir=(9000,0,0)), \
                start=0x5000, end=0x0000, length=5000, attack_length=1000, fade_length=1000))
            self.id[self.nefx] = sdl2.SDL_HapticNewEffect(self.haptic, self.efx[self.nefx])
            self.nefx += 1

        if (supported & sdl2.SDL_HAPTIC_CONSTANT):
            print("   effect", self.nefx, "Constant Force")
            self.efx[self.nefx] = sdl2.SDL_HapticEffect(type=sdl2.SDL_HAPTIC_CONSTANT, constant= \
                sdl2.SDL_HapticConstant(type=sdl2.SDL_HAPTIC_CONSTANT, direction=sdl2.SDL_HapticDirection(type=sdl2.SDL_HAPTIC_CARTESIAN, dir=(1,0,0)), \
                length=5000, level=0x4000, attack_length=1000, fade_length=1000))
            self.id[self.nefx] = sdl2.SDL_HapticNewEffect(self.haptic, self.efx[self.nefx])
            self.nefx += 1
    
    def start_haptic(self,haptic_number):
        sdl2.SDL_HapticRunEffect(self.haptic, self.id[haptic_number], 1)

    def stop_haptic(self):
        sdl2.SDL_HapticStopAll(self.haptic)


In [2]:
import time
h = Haptic()

Trying to find haptics
Found 0 : b'LS-USBMX1/2/3 Steering\xc2\xa1\xc2\xad'
Using device 0
   effect 0 Sine Wave
   effect 1 Triangle
   effect 2 Sawtooth Up
   effect 3 Sawtooth Down
   effect 4 Ramp
   effect 5 Constant Force


In [6]:
h.start_haptic(5)
time.sleep(3)
h.stop_haptic()